## Transactions

In [1]:
import os
import sqlite3

In [2]:
# Create the connection and enable foreign keys on it.
db = sqlite3.connect("transactions.db")

In [3]:
cursor = db.cursor()

#### Create tables

In [ ]:
# Create the table for adresses.
cursor.execute(
    """
    CREATE TABLE Adressen (
        AdresId INTEGER PRIMARY KEY,
        Postcode TEXT NOT NULL,
        Huisnummer INTEGER NOT NULL
    );
    """
)

In [ ]:
# Create the customer table which references the adresses table.
cursor.execute(
    """
    CREATE TABLE Klanten (
        KlantId INTEGER PRIMARY KEY,
        LeverAdresId INTEGER NOT NULL,

        Voornaam TEXT NOT NULL,
        Achternaam TEXT NOT NULL,
        GeboorteDatum DATE,

        FOREIGN KEY(LeverAdresId) REFERENCES Adressen(AdresId)
    );
    """
)

### Inserting data

In [ ]:
# Default is to automatically commit changes.
cursor.execute(
    """
    INSERT INTO Adressen
        (Postcode, Huisnummer)
    VALUES
        ('1111 AA', 1)
    ;
    """
)

In [ ]:
# Seems like data was inserted...
cursor.execute("SELECT * FROM Adressen").fetchall()

In [8]:
# But... no data visible when checking with DB Browser.

In [9]:
# Commit the changes to the database file.
db.commit()

In [10]:
# Check again with DB Browser.

### Explicit transactions

In [ ]:
# Explicitly start a transaction.
cursor.execute("BEGIN;")

In [ ]:
# Insert address first so we can refer to it.
cursor.execute(
    """
    INSERT INTO Adressen
        (Postcode, Huisnummer)
    VALUES
        ('2222 BB', 2)
    ;
    """
)

In [ ]:
# Get the automatically generated address ID.
address_id = cursor.lastrowid
address_id

In [ ]:
# Create the customer with the generated address ID.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum, LeverAdresId)
    VALUES
        ('Ingrid', 'Jansen', '1984-09-06', ?)
    ;
    """,
    [address_id]
)

In [15]:
# Verify nothing was actually inserted using DB Browser.

In [ ]:
# Everything was OK; commit changes to the database
# Alternative: db.commit()
cursor.execute("COMMIT;")

In [17]:
# Verify data was inserted to file using DB Browser.

In [ ]:
# Start a new transaction.
cursor.execute("BEGIN;")

In [ ]:
# Insert another address and get the ID.
cursor.execute(
    """
    INSERT INTO Adressen
        (Postcode, Huisnummer)
    VALUES
        ('3333 CC', 3)
    ;
    """
)
address_id = cursor.lastrowid
address_id

In [ ]:
# Try to insert an invalid customer record...
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum, LeverAdresId)
    VALUES
        (NULL, 'Knol', '1957-08-23', ?)
    ;
    """,
    [address_id]
)

In [ ]:
# Roll back the pending address insert statement.
# Alternative: db.rollback()
cursor.execute("ROLLBACK;")

In [ ]:
# Note that adress insertion was reverted.
db.execute("SELECT * FROM Adressen").fetchall()


### Handle exceptions

In [23]:
# Define some data.
customers = [
    {
        "Voornaam": "Mark",
        "Achternaam": "Vos",
        "GeboorteDatum": "1977-07-27",
        "Adres": {
            "Postcode": "1234 AB",
            "Huisnummer": 1,
            "Toevoeging": None,
        }
    },
    {
        "Voornaam": None,
        "Achternaam": "Jansen",
        "GeboorteDatum": "1995-05-23",
        "Adres": {
            "Postcode": "4567 CD",
            "Huisnummer": 2,
            "Toevoeging": None,
        }
    },
]

In [24]:
# Define templates for the queries.
address_query = """
    INSERT INTO Adressen
    (Postcode, Huisnummer)
    VALUES (:Postcode, :Huisnummer);
"""

customer_query = """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum, LeverAdresId)
    VALUES (:Voornaam, :Achternaam, :GeboorteDatum, :LeverAdresId);
"""

In [ ]:
for customer in customers:
    # Use try ... except to handle integrity errors.
    try:
        cursor.execute(address_query, customer["Adres"])
        customer["LeverAdresId"] = cursor.lastrowid

        cursor.execute(customer_query, customer)
        db.commit()

        print(
            f"Succesfully inserted: {customer['Voornaam']} {customer['Achternaam']}"
        )

    except sqlite3.IntegrityError as error:
        print(
            f"Could not insert: {customer['Voornaam']} {customer['Achternaam']}"
        )
        db.rollback()


In [26]:
# Clean up.
cursor.close()
db.close()
os.remove("transactions.db")